# Exploring the Riot API

#### Finding out match statistics between the top 0.1% of players vs the bottom 50% of players

##### Will have to find data from each rank/tier first, look at each user id and pull data from each individual user

In [ ]:
pip install ratelimit

In [1]:
from pandas import json_normalize
import requests
import pandas as pd
import numpy as np
import time
from ratelimit import limits, sleep_and_retry

In [16]:
my_api = 'API KEY HERE'

In [107]:
chall = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + my_api
req = requests.get(chall)
challenger_df = json_normalize(req.json()['entries'])
challenger_df = challenger_df.sort_values('leaguePoints',ascending = False)


challenger_df.iloc[0] # This is the top player in all of the na1 region

first_place = challenger_df['summonerId'].iloc[0] # Here we're singling out his unique summonerId
first_place

# Challenger has 300 players which holds true for the API

'j3PwW-HH3eCGMYN24t9FKpYMHIAPy1j0dZXjNs9LEwSWU5RF'

In [5]:
grand = 'https://na1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + my_api
req = requests.get(grand)
grandmaster_df = json_normalize(req.json()['entries'])
grandmaster_df = grandmaster_df.sort_values('leaguePoints',ascending=False)
grandmaster_df = grandmaster_df.drop(['summonerId'],axis=1)

grandmaster_df

# Grandmaster rank includes the rest until the top 1000 players which is how many rows we have here

,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
600,Allorim,625,I,132,119,False,False,True,True
536,Yasinn,616,I,235,211,True,False,False,True
100,Familiar,607,I,154,134,True,False,False,False
593,shadow reaIm,607,I,99,73,False,False,True,False
363,BEST POUTINE QC,606,I,261,242,False,False,True,True
...,...,...,...,...,...,...,...,...,...
498,Qube,217,I,243,221,False,False,True,False
143,GKyrios,198,I,377,379,False,False,False,False
592,Piloto275LV,192,I,61,56,False,False,True,False
546,INANI,189,I,235,227,False,False,True,False


In [6]:
master = 'https://na1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + my_api
req = requests.get(master)
master_df = json_normalize(req.json()['entries'])
master_df = master_df.sort_values('leaguePoints',ascending=False)
master_df = master_df.drop(['summonerId'],axis=1)

master_df

# master rank includes the rest of the top ~0.2% of players

,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
50,distelfinks,317,I,348,335,False,False,True,False
628,the bob killer 2,295,I,109,89,False,False,True,False
553,Sõûlż,276,I,134,126,False,False,True,True
446,TeéKay,276,I,513,476,False,False,True,True
291,ln2you,276,I,85,56,False,False,True,False
...,...,...,...,...,...,...,...,...,...
1690,Krythex,0,I,135,134,False,False,True,False
523,TF Espada,0,I,44,47,False,False,False,False
1693,sketchdreamss,0,I,90,67,False,False,False,False
1698,McTrump,0,I,102,96,False,False,True,False


In [7]:
# using the variables we called above, add all top ranks to "top_players"
r = requests.get(chall)
top_players = pd.DataFrame(r.json())
r = requests.get(grand)
top_players = pd.concat([top_players, pd.DataFrame(r.json())], axis=0)
r = requests.get(master)
top_players = pd.concat([top_players, pd.DataFrame(r.json())], axis=0)
top_players

,tier,leagueId,queue,name,entries
0,CHALLENGER,2bd81059-a0ce-33d4-9fb0-86977c851f64,RANKED_SOLO_5x5,Alistar's Captains,{'summonerId': '2je6pbrCAXwUa0ZAak_Ua71IhP-Cb-...
1,CHALLENGER,2bd81059-a0ce-33d4-9fb0-86977c851f64,RANKED_SOLO_5x5,Alistar's Captains,{'summonerId': 'FOuFQnnCDVTpZbkiyuVvOeVR-nw3ah...
2,CHALLENGER,2bd81059-a0ce-33d4-9fb0-86977c851f64,RANKED_SOLO_5x5,Alistar's Captains,{'summonerId': 'omvy7PwUl7bTUh2CvZRUzzcs4AubqP...
3,CHALLENGER,2bd81059-a0ce-33d4-9fb0-86977c851f64,RANKED_SOLO_5x5,Alistar's Captains,{'summonerId': 'zPZmO0-Nl4VKJpnERtbtKgmgtjCy50...
4,CHALLENGER,2bd81059-a0ce-33d4-9fb0-86977c851f64,RANKED_SOLO_5x5,Alistar's Captains,{'summonerId': 'DtDJrV6zfIoRzICDNF543Z6dC2dK3c...
...,...,...,...,...,...
1923,MASTER,1a3db499-99cb-32f0-b393-e1dd2e1467ea,RANKED_SOLO_5x5,Lulu's Battlemasters,{'summonerId': '9pfrKzXDLzi1tYEUcPlDspAaER5qaW...
1924,MASTER,1a3db499-99cb-32f0-b393-e1dd2e1467ea,RANKED_SOLO_5x5,Lulu's Battlemasters,{'summonerId': 'RGTleGalkUIOFXvR5WRddkCzmq4-Rb...
1925,MASTER,1a3db499-99cb-32f0-b393-e1dd2e1467ea,RANKED_SOLO_5x5,Lulu's Battlemasters,{'summonerId': 'ovDG3_ACjUFW-SO6-6H8RTtlpzG3p6...
1926,MASTER,1a3db499-99cb-32f0-b393-e1dd2e1467ea,RANKED_SOLO_5x5,Lulu's Battlemasters,{'summonerId': '4IUiGb9dCDjqI6JpBUZ5CKu4ilPTuC...


In [ ]:
top_players['entries']
# Riot API calls their data within each summoner id entries as a dictionary

### The pertinent information is within a nested dictionary which we will need to grab (entries)

In [8]:
top_players.reset_index(inplace=True)
top_players_entries = pd.DataFrame(dict(top_players["entries"])).T # Will allow use to create a df using the nested dict
# Once we've grabbed that data we can concatenate our initial df with the new "entries" df
top_players = pd.concat([top_players, top_players_entries], axis=1)

top_players = top_players.drop(["index", "queue", "name", "leagueId","entries","rank"], axis=1)
top_players

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,CHALLENGER,2je6pbrCAXwUa0ZAak_Ua71IhP-Cb-EQT14wq4KaFpcaL3w,VX REVERSE6,748,409,338,False,False,False,True
1,CHALLENGER,FOuFQnnCDVTpZbkiyuVvOeVR-nw3ahiq5P7oFpvb7tpQnP...,Grey Tulip,752,92,61,False,False,False,False
2,CHALLENGER,omvy7PwUl7bTUh2CvZRUzzcs4AubqPcBxIRP8UHxV-i4QWM,Chase,612,167,128,False,False,True,False
3,CHALLENGER,zPZmO0-Nl4VKJpnERtbtKgmgtjCy50z_yTfSgs0KEZA0Iq...,quantumfan,640,220,208,True,False,False,False
4,CHALLENGER,DtDJrV6zfIoRzICDNF543Z6dC2dK3c7z6UQQXXEH-RE-rKk,5fire,849,148,134,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
2923,MASTER,9pfrKzXDLzi1tYEUcPlDspAaER5qaWen5Buvq6t7N1kME1tv,Pöp2o2,0,197,152,False,False,False,False
2924,MASTER,RGTleGalkUIOFXvR5WRddkCzmq4-RbLphU6zG2ODZben764,Klexo,168,210,206,True,False,False,False
2925,MASTER,ovDG3_ACjUFW-SO6-6H8RTtlpzG3p60VspB8DCWTx67LoUI,MightyMongoose,25,48,34,False,False,True,True
2926,MASTER,4IUiGb9dCDjqI6JpBUZ5CKu4ilPTuC7H1VCvGK1ueMWKhNA,Yuę,1,111,65,False,False,True,False


In [10]:
@sleep_and_retry
@limits(calls=5, period=10)
def check_limit():
# Empty function just to check for calls to API 
    return
## Ended up not needing this because I opted for a time.sleep within the for loop instead. (This takes forever)
# @limits wrapper was also not working as intended.

In [68]:
# @limits period will be in seconds. (default is 900s or 15 min)
# Riot API allows for 20 requests every 1 second & 100 requests every 2 minutes

#FIFTEEN_MINUTES = 900
THREE_MINUTES = 180

@sleep_and_retry
@limits(calls=5, period=THREE_MINUTES) # wrap function so that we don't make too many calls to the api.
def call_api(df):
    count = 0 # If rate-limit is exceeded we can see how many api calls we made before hitting limit
    for i, summoner_id in enumerate(df["summonerId"]):
        time.sleep(1.5)
        api_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/" + summoner_id + "?api_key=" + my_api
        r = requests.get(api_url)
        count += 1
        while r.status_code!=200: 
            print(count) # print when there's an unsuccessful api call
            time.sleep(30)
            r = requests.get(api_url)
        account_id = r.json()["accountId"]
        df.iloc[i, -1] = account_id
    return df


#### Testing api calls function with a smaller dataframe of just challenger tier players

kept hitting the rate limit - need to use a sleep and retry or limit the function entirely with every iteration

In [ ]:
req = requests.get(chall)
challenger_df2 = pd.DataFrame(req.json())

challenger_df2

In [ ]:
challenger_df2.reset_index(inplace=True)
challenger_df2_entries = pd.DataFrame(dict(challenger_df2["entries"])).T
challenger_df2 = pd.concat([challenger_df2, challenger_df2_entries], axis=1)

challenger_df2 = challenger_df2.drop(["index", "queue", "name", "leagueId","entries","rank"], axis=1)
challenger_df2

In [ ]:
challenger_df2.insert(loc=10,column='accountId',value=np.nan)
challenger_df2

### Creating a new column in the "top_players" df with unique account id's to access other parts of the Riot API

In [9]:
top_players.insert(loc=10,column='accountId',value=np.nan)
top_players

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,accountId
0,CHALLENGER,2je6pbrCAXwUa0ZAak_Ua71IhP-Cb-EQT14wq4KaFpcaL3w,VX REVERSE6,748,409,338,False,False,False,True,NaN
1,CHALLENGER,FOuFQnnCDVTpZbkiyuVvOeVR-nw3ahiq5P7oFpvb7tpQnP...,Grey Tulip,752,92,61,False,False,False,False,NaN
2,CHALLENGER,omvy7PwUl7bTUh2CvZRUzzcs4AubqPcBxIRP8UHxV-i4QWM,Chase,612,167,128,False,False,True,False,NaN
3,CHALLENGER,zPZmO0-Nl4VKJpnERtbtKgmgtjCy50z_yTfSgs0KEZA0Iq...,quantumfan,640,220,208,True,False,False,False,NaN
4,CHALLENGER,DtDJrV6zfIoRzICDNF543Z6dC2dK3c7z6UQQXXEH-RE-rKk,5fire,849,148,134,True,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2923,MASTER,9pfrKzXDLzi1tYEUcPlDspAaER5qaWen5Buvq6t7N1kME1tv,Pöp2o2,0,197,152,False,False,False,False,NaN
2924,MASTER,RGTleGalkUIOFXvR5WRddkCzmq4-RbLphU6zG2ODZben764,Klexo,168,210,206,True,False,False,False,NaN
2925,MASTER,ovDG3_ACjUFW-SO6-6H8RTtlpzG3p60VspB8DCWTx67LoUI,MightyMongoose,25,48,34,False,False,True,True,NaN
2926,MASTER,4IUiGb9dCDjqI6JpBUZ5CKu4ilPTuC7H1VCvGK1ueMWKhNA,Yuę,1,111,65,False,False,True,False,NaN


#### Call the function to add accountId's

In [14]:
call_api(top_players)

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,accountId
0,CHALLENGER,2je6pbrCAXwUa0ZAak_Ua71IhP-Cb-EQT14wq4KaFpcaL3w,VX REVERSE6,748,409,338,False,False,False,True,Qfiv008XrYljGBrF_nC9UMyb6qEN7R-VDDiQtcq_DEzrx0A
1,CHALLENGER,FOuFQnnCDVTpZbkiyuVvOeVR-nw3ahiq5P7oFpvb7tpQnP...,Grey Tulip,752,92,61,False,False,False,False,K4RFYdy-tcYNt6fwE_WY2AuvU9k909rNG-NqM_h2UJrxQ5...
2,CHALLENGER,omvy7PwUl7bTUh2CvZRUzzcs4AubqPcBxIRP8UHxV-i4QWM,Chase,612,167,128,False,False,True,False,yl_8KSsY1MrRjhV_9to5wPVUEEtstsgAIN61pnQLDhRY6g
3,CHALLENGER,zPZmO0-Nl4VKJpnERtbtKgmgtjCy50z_yTfSgs0KEZA0Iq...,quantumfan,640,220,208,True,False,False,False,Fc3zh1jfJukEx7BqOMkhpu-q7kmhsX0mpx6VilHdttZeyG...
4,CHALLENGER,DtDJrV6zfIoRzICDNF543Z6dC2dK3c7z6UQQXXEH-RE-rKk,5fire,849,148,134,True,False,False,False,gTzJPVWwxropWxtofKtvumkWFk6FmBYABUPc6rUsv_nwTA
...,...,...,...,...,...,...,...,...,...,...,...
2923,MASTER,9pfrKzXDLzi1tYEUcPlDspAaER5qaWen5Buvq6t7N1kME1tv,Pöp2o2,0,197,152,False,False,False,False,9llyTX62G5cTLM5YXgU1URkaNk5xBxmt9mvdiTL0EDJ3FX...
2924,MASTER,RGTleGalkUIOFXvR5WRddkCzmq4-RbLphU6zG2ODZben764,Klexo,168,210,206,True,False,False,False,L9JqYEV9U2pJDWLeO32bwFBVIdCTV9f_J1FxlYEDvAMPD_o
2925,MASTER,ovDG3_ACjUFW-SO6-6H8RTtlpzG3p60VspB8DCWTx67LoUI,MightyMongoose,25,48,34,False,False,True,True,2IPbXGIBTBNecZEoZzF3TFKdjE2rPpgJblfewjo3nLht3w
2926,MASTER,4IUiGb9dCDjqI6JpBUZ5CKu4ilPTuC7H1VCvGK1ueMWKhNA,Yuę,1,111,65,False,False,True,False,0Blgz0zmVdGvrm24QjjyBL-R294Lxf-ptTJFRC0LsPwMUwA


In [15]:
top_players.to_csv("Top_League_Players_with_accountId.csv")

In [48]:
iron = 'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/IRON/IV?page=1&api_key=' + my_api
req = requests.get(iron)
iron_df = json_normalize(req.json())
iron_df = iron_df.sort_values('leaguePoints',ascending=False)


iron_df

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
53,236bf7df-c8c0-44b0-afff-e05bd6c88657,RANKED_SOLO_5x5,IRON,IV,OCFbJf3eR3XIywZvHqnNxE11gUHse1PHLyJXAaCVZOByMrfD,6 F00T D0NGER,99,16,80,False,False,False,False
65,0c2d3c96-bd58-4ee3-ade0-ebddf160201e,RANKED_SOLO_5x5,IRON,IV,qZ0X1vuPE1Tsp6j980giyb-_rX-dxdLK10qJOjssOHs7CWPI,Equities433,98,28,40,False,False,False,False
4,53bc3c51-2262-4ef4-a987-c31cf43eb471,RANKED_SOLO_5x5,IRON,IV,OkeRxipXFznCvBTtb3VBS0qz_QmkF98ldEWVtRfRE2MLb1mD,Brawler319,96,52,70,False,False,False,False
183,1097cb71-7676-434d-8b6d-4f8cbc26cdb5,RANKED_SOLO_5x5,IRON,IV,r08fKNEP_cDPW1x-hAf3LVPVeGP4NETyj1m2UALTLwS8IRLd,calistaaaaa,93,71,98,False,False,False,False
145,ba3991ab-20c6-4cf4-a4a5-5e10a769225d,RANKED_SOLO_5x5,IRON,IV,VtlQFw7IrUp5Zl2SZ4hi0CRN5cHRqdaIHdhkECyiaFvgPbz9,CBsnowgath,93,21,82,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2172016a-e802-4cba-99ad-f02306466c1a,RANKED_SOLO_5x5,IRON,IV,uC4zV0xbvkUBiBQg10EWel1bBr2PnMX1jjPo__uMzw_5a0Zd,MistaStealYoRole,0,34,81,False,False,False,False
94,20c3f643-5111-4eb8-8a32-cd211b332e41,RANKED_SOLO_5x5,IRON,IV,kSM-_hCM-OPclQIjYYZrVr07l-89txAU4fk-zXggjH_vYe1m,sludymochi,0,4,19,False,False,False,False
90,33aa360d-d3bb-433a-9e56-7dfc385e7679,RANKED_SOLO_5x5,IRON,IV,oevXt38zsJ8uVa8LorF96kBUHXRxT0048pSH7ry_L_NqldGo,AlYaCOOKIE,0,31,44,False,False,False,False
88,73e80b6a-67d9-467b-af2e-ddc0c64be79f,RANKED_SOLO_5x5,IRON,IV,KAkpK18yqLJ-DB0bCBtylAKShaxpVxh0XZUCFz3AGB9AgMmS,AnuhasChamika,0,45,88,False,False,False,False


In [49]:
# 11 pages of users within iron 4
iron = 'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/IRON/IV?page=1&api_key=' + my_api
r = requests.get(iron)
iron_df = pd.DataFrame(r.json())
for i in range(2,12):
    i = str(i)
    iron = 'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/IRON/IV?page=' + i + '&api_key=' + my_api
    r = requests.get(iron)
    iron_df = pd.concat([iron_df, pd.DataFrame(r.json())], axis=0)
    
iron_df

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,eb171226-3f8b-4bc7-b9a2-7469afba70cb,RANKED_SOLO_5x5,IRON,IV,UMAoSqc4z5WQzP2a4VgUIy-HemBff7whz8428vfVHvstU711,Zparkles,60,8,32,False,False,False,False
1,2172016a-e802-4cba-99ad-f02306466c1a,RANKED_SOLO_5x5,IRON,IV,uC4zV0xbvkUBiBQg10EWel1bBr2PnMX1jjPo__uMzw_5a0Zd,MistaStealYoRole,0,34,81,False,False,False,False
2,34193a87-37d1-4484-8e6a-f0e378efe66e,RANKED_SOLO_5x5,IRON,IV,CyN8UvUeugpUCExzMDv9EEQ7y_Ys1dyeAUg3yarKKff6paX6,InexactFall81,62,30,43,False,False,False,False
3,ab699b00-a145-4499-8b01-11b89cc8152e,RANKED_SOLO_5x5,IRON,IV,wWJbbnPsEu9fc5AvmZJJ2XQML06EW84fW--ieaHWHrFUZs-_,YOURNIMMERBOY,32,68,95,False,False,False,False
4,53bc3c51-2262-4ef4-a987-c31cf43eb471,RANKED_SOLO_5x5,IRON,IV,OkeRxipXFznCvBTtb3VBS0qz_QmkF98ldEWVtRfRE2MLb1mD,Brawler319,96,52,70,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2e20dade-3a33-4923-bbd3-a3ca97a2239b,RANKED_SOLO_5x5,IRON,IV,6jnefr7NNB6B3k3TFH3AWWgb6OKIBovhr8p91_ugBx3fr50,do and die,89,541,595,False,False,False,False
65,229abb8d-7973-4fa6-a416-b2e3d1cadc70,RANKED_SOLO_5x5,IRON,IV,zfsSJCVuEdh3nk2C-yJ362t67BFnXGq02xuV7SH-8IFsI0M,Sinfulangelxxxx,0,38,60,False,False,False,False
66,9d3dec88-2edb-4db5-a4e7-e5803fa64c5b,RANKED_SOLO_5x5,IRON,IV,oPe__1kn4395o_QEb6KkzcFXOUdMBnmqhyDYC2t-W9oHvKk,ElessaMarie2011,60,3,15,False,False,False,False
67,4bf60246-2bab-408b-af08-519bf5de55e2,RANKED_SOLO_5x5,IRON,IV,ZoEgGzC7CYfwIEkcGfMsHqMTiSTZTQgMkCJ_ol4_QlyIKpM,poche bleu,28,2,13,False,False,False,False


In [50]:
for i in range(1,24): # 23 pages of userid's within iron 3
    i = str(i)
    iron = 'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/IRON/III?page=' + i + '&api_key=' + my_api
    r = requests.get(iron)
    iron_df = pd.concat([iron_df, pd.DataFrame(r.json())], axis=0)
    
iron_df

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,eb171226-3f8b-4bc7-b9a2-7469afba70cb,RANKED_SOLO_5x5,IRON,IV,UMAoSqc4z5WQzP2a4VgUIy-HemBff7whz8428vfVHvstU711,Zparkles,60,8,32,False,False,False,False
1,2172016a-e802-4cba-99ad-f02306466c1a,RANKED_SOLO_5x5,IRON,IV,uC4zV0xbvkUBiBQg10EWel1bBr2PnMX1jjPo__uMzw_5a0Zd,MistaStealYoRole,0,34,81,False,False,False,False
2,34193a87-37d1-4484-8e6a-f0e378efe66e,RANKED_SOLO_5x5,IRON,IV,CyN8UvUeugpUCExzMDv9EEQ7y_Ys1dyeAUg3yarKKff6paX6,InexactFall81,62,30,43,False,False,False,False
3,ab699b00-a145-4499-8b01-11b89cc8152e,RANKED_SOLO_5x5,IRON,IV,wWJbbnPsEu9fc5AvmZJJ2XQML06EW84fW--ieaHWHrFUZs-_,YOURNIMMERBOY,32,68,95,False,False,False,False
4,53bc3c51-2262-4ef4-a987-c31cf43eb471,RANKED_SOLO_5x5,IRON,IV,OkeRxipXFznCvBTtb3VBS0qz_QmkF98ldEWVtRfRE2MLb1mD,Brawler319,96,52,70,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,40acf58a-cdba-4309-88e1-3adeee3cec5f,RANKED_SOLO_5x5,IRON,III,1pukwoTcv0nhu-TEdB-sRy6iOQ2yCxaim-MISeeiUl6BsBY,bambque,43,84,112,False,False,False,False
64,75bb9ad8-c8de-4354-b122-09de52d40024,RANKED_SOLO_5x5,IRON,III,ZBBwXQXK0-ypHE3TeUId02YsBM7KxuqfykgnDWZJW0-SU9Q,voldo514,47,3,7,False,False,False,False
65,764e041a-deac-489d-ad6d-310ff3c99c2f,RANKED_SOLO_5x5,IRON,III,jYTYsLmlK874gCCXC920aIbRBkDbVeVH60zNXXBkyKWrico,pinkhandsome,47,5,8,False,False,False,False
66,cfb292c9-91c9-4610-8545-957487c9f7a7,RANKED_SOLO_5x5,IRON,III,3qZpHkpWy5OUC3YRabZGIKa8slhyNlnhJkBK-WsQ4HW0uaQ,JAIR293,43,82,96,False,False,False,False


In [38]:
for i in range(1,51): # 50 pages of users within iron 2
    time.sleep(5)
    i = str(i)
    iron = 'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/IRON/II?page=' + i + '&api_key=' + my_api
    r = requests.get(iron)
    iron_df = pd.concat([iron_df, pd.DataFrame(r.json())], axis=0)
    
iron_df

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,eb171226-3f8b-4bc7-b9a2-7469afba70cb,RANKED_SOLO_5x5,IRON,IV,UMAoSqc4z5WQzP2a4VgUIy-HemBff7whz8428vfVHvstU711,Zparkles,60,8,32,False,False,False,False
1,2172016a-e802-4cba-99ad-f02306466c1a,RANKED_SOLO_5x5,IRON,IV,uC4zV0xbvkUBiBQg10EWel1bBr2PnMX1jjPo__uMzw_5a0Zd,MistaStealYoRole,0,34,81,False,False,False,False
2,34193a87-37d1-4484-8e6a-f0e378efe66e,RANKED_SOLO_5x5,IRON,IV,CyN8UvUeugpUCExzMDv9EEQ7y_Ys1dyeAUg3yarKKff6paX6,InexactFall81,62,30,43,False,False,False,False
3,ab699b00-a145-4499-8b01-11b89cc8152e,RANKED_SOLO_5x5,IRON,IV,wWJbbnPsEu9fc5AvmZJJ2XQML06EW84fW--ieaHWHrFUZs-_,YOURNIMMERBOY,32,68,95,False,False,False,False
4,53bc3c51-2262-4ef4-a987-c31cf43eb471,RANKED_SOLO_5x5,IRON,IV,OkeRxipXFznCvBTtb3VBS0qz_QmkF98ldEWVtRfRE2MLb1mD,Brawler319,96,52,70,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,8f72a791-de45-4fbb-8106-11fe6a5e44e1,RANKED_SOLO_5x5,IRON,II,gRzHe0-QzTcAGkGdeceCP8Ve4RSrlczrD-CN1BMHJT-UYt4,TygerFlash,55,26,29,False,False,False,False
103,cb9fa298-02cf-439e-b39c-47a09b89992d,RANKED_SOLO_5x5,IRON,II,CoYFwfzrWJLcWVeaen8PZaFiH1VRHIom85ASgdnAb62KLAA,xLOLx,46,12,15,False,False,False,False
104,301e1bb4-6381-415d-b09f-0a049e3d593d,RANKED_SOLO_5x5,IRON,II,BZ5J-czW0hPCGxkMhcKdOOkOZdepFs3ytWWNQDoWsURDPLQ,RetroYaYa,77,5,10,False,False,False,False
105,ba49eef5-f2a5-4551-86df-106b9f4f3670,RANKED_SOLO_5x5,IRON,II,8SpQwfpH1vmpy-N_SZOaYZliNLvNL6VeAy7Ci6Hz0p28s_M,djdylan1016,69,29,47,False,False,False,False


### iron_df with this addition is 35,000+ columns which is far too large to grab info from the api in a time efficient manner. Will use iron divisions III and IV as they will have 6600+ and is a little bit more manageable

In [39]:
for i in range(1,91): # 90 pages of users within iron 1
    time.sleep(5)
    i = str(i)
    iron = 'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/IRON/I?page=' + i + '&api_key=' + my_api
    r = requests.get(iron)
    iron_df = pd.concat([iron_df, pd.DataFrame(r.json())], axis=0)
    
iron_df

# iron_df with this addition is 35,000+ columns which is far too large to grab info from the api in a time efficient manner.
# will use iron divisions III and IV as they will have 6600+ and is a little bit more manageable

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,miniSeries
0,eb171226-3f8b-4bc7-b9a2-7469afba70cb,RANKED_SOLO_5x5,IRON,IV,UMAoSqc4z5WQzP2a4VgUIy-HemBff7whz8428vfVHvstU711,Zparkles,60,8,32,False,False,False,False,NaN
1,2172016a-e802-4cba-99ad-f02306466c1a,RANKED_SOLO_5x5,IRON,IV,uC4zV0xbvkUBiBQg10EWel1bBr2PnMX1jjPo__uMzw_5a0Zd,MistaStealYoRole,0,34,81,False,False,False,False,NaN
2,34193a87-37d1-4484-8e6a-f0e378efe66e,RANKED_SOLO_5x5,IRON,IV,CyN8UvUeugpUCExzMDv9EEQ7y_Ys1dyeAUg3yarKKff6paX6,InexactFall81,62,30,43,False,False,False,False,NaN
3,ab699b00-a145-4499-8b01-11b89cc8152e,RANKED_SOLO_5x5,IRON,IV,wWJbbnPsEu9fc5AvmZJJ2XQML06EW84fW--ieaHWHrFUZs-_,YOURNIMMERBOY,32,68,95,False,False,False,False,NaN
4,53bc3c51-2262-4ef4-a987-c31cf43eb471,RANKED_SOLO_5x5,IRON,IV,OkeRxipXFznCvBTtb3VBS0qz_QmkF98ldEWVtRfRE2MLb1mD,Brawler319,96,52,70,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2d04145b-83f2-4cc3-88c2-1cc45447798d,RANKED_SOLO_5x5,IRON,I,Do-Ffu83lnGmnqsh2DcsRQDUmxwSKq46GBdE9pQ8bYEpDS0,PinkiyPie,61,11,14,False,False,False,False,NaN
92,07e75be2-18c7-4123-be97-a3365c34342a,RANKED_SOLO_5x5,IRON,I,PpzG_wBB8SbiuudCL4GkMjOePGpGdNqG2cG--p0rPZt5Q5w,CatchULackin,43,7,10,False,False,False,False,NaN
93,7c7016c2-c2b0-44c7-981f-ec404d534d77,RANKED_SOLO_5x5,IRON,I,qvlO-SJ3QaL13UMewlwPwTIdoCDj3HTPR-_v54A2O0O6Wk0,Keekor,85,4,7,False,False,False,False,NaN
94,c4d5f561-c69b-4557-adda-e09e07985581,RANKED_SOLO_5x5,IRON,I,8doDGCQ_UefhziXtJ7LHIBI8vD3tPLpT0-TLYBY6riXH-2g,IsaacFen,63,50,47,False,False,False,False,NaN


In [56]:
iron_df.insert(loc=13,column='accountId',value=np.nan) # insert NaN column at the end, 13 total columns in this df
iron_df = iron_df.drop(['queueType'],axis=1)
iron_df

,leagueId,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,accountId
0,eb171226-3f8b-4bc7-b9a2-7469afba70cb,IRON,IV,UMAoSqc4z5WQzP2a4VgUIy-HemBff7whz8428vfVHvstU711,Zparkles,60,8,32,False,False,False,False,NaN
1,2172016a-e802-4cba-99ad-f02306466c1a,IRON,IV,uC4zV0xbvkUBiBQg10EWel1bBr2PnMX1jjPo__uMzw_5a0Zd,MistaStealYoRole,0,34,81,False,False,False,False,NaN
2,34193a87-37d1-4484-8e6a-f0e378efe66e,IRON,IV,CyN8UvUeugpUCExzMDv9EEQ7y_Ys1dyeAUg3yarKKff6paX6,InexactFall81,62,30,43,False,False,False,False,NaN
3,ab699b00-a145-4499-8b01-11b89cc8152e,IRON,IV,wWJbbnPsEu9fc5AvmZJJ2XQML06EW84fW--ieaHWHrFUZs-_,YOURNIMMERBOY,32,68,95,False,False,False,False,NaN
4,53bc3c51-2262-4ef4-a987-c31cf43eb471,IRON,IV,OkeRxipXFznCvBTtb3VBS0qz_QmkF98ldEWVtRfRE2MLb1mD,Brawler319,96,52,70,False,False,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,40acf58a-cdba-4309-88e1-3adeee3cec5f,IRON,III,1pukwoTcv0nhu-TEdB-sRy6iOQ2yCxaim-MISeeiUl6BsBY,bambque,43,84,112,False,False,False,False,NaN
64,75bb9ad8-c8de-4354-b122-09de52d40024,IRON,III,ZBBwXQXK0-ypHE3TeUId02YsBM7KxuqfykgnDWZJW0-SU9Q,voldo514,47,3,7,False,False,False,False,NaN
65,764e041a-deac-489d-ad6d-310ff3c99c2f,IRON,III,jYTYsLmlK874gCCXC920aIbRBkDbVeVH60zNXXBkyKWrico,pinkhandsome,47,5,8,False,False,False,False,NaN
66,cfb292c9-91c9-4610-8545-957487c9f7a7,IRON,III,3qZpHkpWy5OUC3YRabZGIKa8slhyNlnhJkBK-WsQ4HW0uaQ,JAIR293,43,82,96,False,False,False,False,NaN


In [69]:
call_api(iron_df)

,leagueId,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,accountId
0,eb171226-3f8b-4bc7-b9a2-7469afba70cb,IRON,IV,UMAoSqc4z5WQzP2a4VgUIy-HemBff7whz8428vfVHvstU711,Zparkles,60,8,32,False,False,False,False,h1wd0rsjBe0kT0_m7CPl38my2yY4VB2LCd-R5Dp7y1UvtL...
1,2172016a-e802-4cba-99ad-f02306466c1a,IRON,IV,uC4zV0xbvkUBiBQg10EWel1bBr2PnMX1jjPo__uMzw_5a0Zd,MistaStealYoRole,0,34,81,False,False,False,False,Dg0pFTotA3wtGwuaC47QHp_f_UJpLVGcVC8LMnaOgYUG5J...
2,34193a87-37d1-4484-8e6a-f0e378efe66e,IRON,IV,CyN8UvUeugpUCExzMDv9EEQ7y_Ys1dyeAUg3yarKKff6paX6,InexactFall81,62,30,43,False,False,False,False,9l8z-dY8MLJwDm_2Z7rqMJ0CDgzklzFCDojML_hXGBbInW...
3,ab699b00-a145-4499-8b01-11b89cc8152e,IRON,IV,wWJbbnPsEu9fc5AvmZJJ2XQML06EW84fW--ieaHWHrFUZs-_,YOURNIMMERBOY,32,68,95,False,False,False,False,WRN991V2W2UXIOL4w4cEBPnnRR_X7F8ykKDUDyT7H3x4l-...
4,53bc3c51-2262-4ef4-a987-c31cf43eb471,IRON,IV,OkeRxipXFznCvBTtb3VBS0qz_QmkF98ldEWVtRfRE2MLb1mD,Brawler319,96,52,70,False,False,False,False,dvBgWthUTTciM_leCBgtUyuVt7Lo4YUdPFkWCbXM-nsT10...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,40acf58a-cdba-4309-88e1-3adeee3cec5f,IRON,III,1pukwoTcv0nhu-TEdB-sRy6iOQ2yCxaim-MISeeiUl6BsBY,bambque,43,84,112,False,False,False,False,pIwVlEtnQOuyZ3itdALXv8ooooR1jbO0UvCe_SDZxgpWqI...
64,75bb9ad8-c8de-4354-b122-09de52d40024,IRON,III,ZBBwXQXK0-ypHE3TeUId02YsBM7KxuqfykgnDWZJW0-SU9Q,voldo514,47,3,7,False,False,False,False,sVBkZf-S4zvN2dGShTUPDS5KWy388U6T-LjRNHwmFe4J2h...
65,764e041a-deac-489d-ad6d-310ff3c99c2f,IRON,III,jYTYsLmlK874gCCXC920aIbRBkDbVeVH60zNXXBkyKWrico,pinkhandsome,47,5,8,False,False,False,False,sRMbzZzQ8AdEEsQd46lgt5wRfjODOcCCPBip9u_UxnyT7n...
66,cfb292c9-91c9-4610-8545-957487c9f7a7,IRON,III,3qZpHkpWy5OUC3YRabZGIKa8slhyNlnhJkBK-WsQ4HW0uaQ,JAIR293,43,82,96,False,False,False,False,oLuFFGE6eHJsCMp3MM_J95aCZMzSsaeJmxY2TFoaZYNpVS...


In [70]:
iron_df.to_csv("iron_league_players_with_accountId.csv")

## At this point we have all of the data from the top players and the bottom players

The top .15% and the bottom .15% of the whole league of legends playerbase (within the NA1 region)

What we can explore from here is more of the Riot API using the unique accountId's that we just grabbed.
What we can use from there is a unique match id that will give us information regarding the participating players and
their corresponding statistics. Thus we'll be able to compare total match time, kills, objectives, even creep kills.

In [97]:
season = str(13) 

match_info_df = pd.DataFrame()
# challenger_df2_entries = pd.DataFrame(dict(challenger_df2["entries"])).T

def call_match_info_api(df):
    count = 0
    for account_id in df["accountId"]:
        time.sleep(1)
        count += 1
        api_url = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + account_id + "?season=" + season + "&api_key=" + my_api
        r = requests.get(api_url)
        while r.status_code!=200:
            print(count)
            time.sleep(30)
            r = requests.get(api_url)
        match_info_df = pd.DataFrame(json_normalize(r.json()["matches"]))
        print(match_info_df)
        match_info_df = pd.concat([match_info_df, pd.DataFrame(json_normalize(r.json()["matches"]))])
    return match_info_df


In [102]:
sample_account_id = 'Qfiv008XrYljGBrF_nC9UMyb6qEN7R-VDDiQtcq_DEzrx0A'
req = requests.get("https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + sample_account_id + "?season=" + season + "&api_key=" + my_api)
match_info_df = json_normalize(req.json()['matches'])

match_info_df

# Initialize match_info dataframe to hold all information regarding the games the specific accountId has played in
# Particularly what we want her is the gameID which we can use to go through the Riot API again 
# for specific details about that match - kills, deaths, time, etc

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,NA1,3904139573,117,420,13,1620849653108,DUO_SUPPORT,BOTTOM
1,NA1,3904134397,105,420,13,1620846644681,DUO_SUPPORT,MID
2,NA1,3904067843,517,420,13,1620842459917,DUO,BOTTOM
3,NA1,3904074638,517,420,13,1620840245664,SOLO,MID
4,NA1,3903064055,64,420,13,1620791887613,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
95,NA1,3876261304,81,450,13,1619055350399,DUO_SUPPORT,NONE
96,NA1,3876147919,157,450,13,1619053762662,DUO,MID
97,NA1,3876134449,38,450,13,1619052428188,DUO,NONE
98,NA1,3876044748,887,430,13,1619044375077,SOLO,MID


In [103]:
match_info_df

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,NA1,3904139573,117,420,13,1620849653108,DUO_SUPPORT,BOTTOM
1,NA1,3904134397,105,420,13,1620846644681,DUO_SUPPORT,MID
2,NA1,3904067843,517,420,13,1620842459917,DUO,BOTTOM
3,NA1,3904074638,517,420,13,1620840245664,SOLO,MID
4,NA1,3903064055,64,420,13,1620791887613,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
95,NA1,3876261304,81,450,13,1619055350399,DUO_SUPPORT,NONE
96,NA1,3876147919,157,450,13,1619053762662,DUO,MID
97,NA1,3876134449,38,450,13,1619052428188,DUO,NONE
98,NA1,3876044748,887,430,13,1619044375077,SOLO,MID


#### Above we can see that there's a new column of interest "gameId"
This column contains the information we need to access the specific "matches" part of the Riot API
All of the games contain a specific gameId which will contain the participants (10 in total)
as well as all of their stats in that game. 

## From here we will just use the #1 player in the region

###### We can create a function to iterate through all of the gameIds to return a full df of statistics, but grabbing statistics is another issue because it is nested within the api call of the gameId

In [129]:
first_place = top_players[top_players['summonerId'] == 'j3PwW-HH3eCGMYN24t9FKpYMHIAPy1j0dZXjNs9LEwSWU5RF']
# Above code allows us to reference the #1 player in the region by his summonerID to grab his accountId
# Which we will need his accountId in order to access the match api
first_place_accountId = first_place['accountId']
first_place_accountId = (first_place_accountId.values[0])

In [135]:
#first_place == 'j3PwW-HH3eCGMYN24t9FKpYMHIAPy1j0dZXjNs9LEwSWU5RF'
req = requests.get("https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/"+ first_place_accountId + "?season=" + season + "&api_key=" + my_api)
first_place_df_match_info_df = json_normalize(req.json()['matches'])
first_place_df_match_info_df

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,NA1,3906836211,412,420,13,1621038735487,DUO_SUPPORT,BOTTOM
1,NA1,3906831803,412,420,13,1621036730877,DUO_SUPPORT,BOTTOM
2,NA1,3906776967,412,420,13,1621034430770,DUO_SUPPORT,BOTTOM
3,NA1,3906772833,412,420,13,1621032266106,DUO_SUPPORT,NONE
4,NA1,3906647084,111,420,13,1621028725117,DUO_SUPPORT,BOTTOM
...,...,...,...,...,...,...,...,...
95,NA1,3885779839,875,420,13,1619676331114,DUO_SUPPORT,NONE
96,NA1,3885693461,222,420,13,1619673981268,DUO_CARRY,BOTTOM
97,NA1,3885604793,875,420,13,1619671201003,DUO_SUPPORT,BOTTOM
98,NA1,3885524545,117,420,13,1619668240438,DUO_SUPPORT,NONE


In [171]:
first_place_game_info_df = pd.DataFrame()
req = requests.get('https://na1.api.riotgames.com/lol/match/v4/matches/3906836211?api_key=RGAPI-356d462f-39ea-42e5-a288-fcc45f8a63ae')
r = req.json()
temp_df = pd.DataFrame(list(r.values()), index=list(r.keys())).T
first_place_game_info_df = pd.concat([first_place_game_info_df, temp_df])

first_place_game_info_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,3906836211,NA1,1621038735487,1500,420,11,13,11.10.374.9538,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [218]:
stats = pd.DataFrame(dict(first_place_game_info_df["participants"])).T

first_place_game_info_df = pd.concat([first_place_game_info_df, stats], axis=1)
first_place_game_info_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities,0,1,2,3,4,5,6,7,8,9
0,3906836211,NA1,1621038735487,1500,420,11,13,11.10.374.9538,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...","{'participantId': 1, 'teamId': 100, 'championI...","{'participantId': 2, 'teamId': 100, 'championI...","{'participantId': 3, 'teamId': 100, 'championI...","{'participantId': 4, 'teamId': 100, 'championI...","{'participantId': 5, 'teamId': 100, 'championI...","{'participantId': 6, 'teamId': 200, 'championI...","{'participantId': 7, 'teamId': 200, 'championI...","{'participantId': 8, 'teamId': 200, 'championI...","{'participantId': 9, 'teamId': 200, 'championI...","{'participantId': 10, 'teamId': 200, 'champion..."


In [203]:
test = [] # testing how to obtain the nested participants data(which holds the statistics data)
test = first_place_game_info_df['participants'].values.tolist() 
# create an empty list and then convert the participants data to a list and set it equal to test
# test currently looks like a nested dictionary within a nested list. == [ [ { } ] ]
test_df = pd.concat([pd.DataFrame(t) for t in test]) # Flatten the list. 
test_df
# Finally able to see a stats column which is what we want.

,participantId,teamId,championId,spell1Id,spell2Id,stats,timeline
0,1,100,26,14,4,"{'participantId': 1, 'win': False, 'item0': 30...","{'participantId': 1, 'creepsPerMinDeltas': {'1..."
1,2,100,875,4,12,"{'participantId': 2, 'win': False, 'item0': 31...","{'participantId': 2, 'creepsPerMinDeltas': {'1..."
2,3,100,79,4,12,"{'participantId': 3, 'win': False, 'item0': 10...","{'participantId': 3, 'creepsPerMinDeltas': {'1..."
3,4,100,234,4,11,"{'participantId': 4, 'win': False, 'item0': 66...","{'participantId': 4, 'creepsPerMinDeltas': {'1..."
4,5,100,21,7,4,"{'participantId': 5, 'win': False, 'item0': 66...","{'participantId': 5, 'creepsPerMinDeltas': {'1..."
5,6,200,412,4,14,"{'participantId': 6, 'win': True, 'item0': 206...","{'participantId': 6, 'creepsPerMinDeltas': {'1..."
6,7,200,164,4,12,"{'participantId': 7, 'win': True, 'item0': 315...","{'participantId': 7, 'creepsPerMinDeltas': {'1..."
7,8,200,245,4,12,"{'participantId': 8, 'win': True, 'item0': 203...","{'participantId': 8, 'creepsPerMinDeltas': {'1..."
8,9,200,51,4,7,"{'participantId': 9, 'win': True, 'item0': 105...","{'participantId': 9, 'creepsPerMinDeltas': {'1..."
9,10,200,25,11,4,"{'participantId': 10, 'win': True, 'item0': 30...","{'participantId': 10, 'creepsPerMinDeltas': {'..."


In [213]:
stats = []
stats = test_df['stats'].values.tolist()
stats
# Using a similar strategy as above to create a list of the dictionary of "stats"

[{'participantId': 1,
  'win': False,
  'item0': 3067,
  'item1': 3853,
  'item2': 2065,
  'item3': 1082,
  'item4': 3158,
  'item5': 3108,
  'item6': 3364,
  'kills': 3,
  'deaths': 3,
  'assists': 4,
  'largestKillingSpree': 0,
  'largestMultiKill': 1,
  'killingSprees': 0,
  'longestTimeSpentLiving': 1083,
  'doubleKills': 0,
  'tripleKills': 0,
  'quadraKills': 0,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 25604,
  'magicDamageDealt': 22187,
  'physicalDamageDealt': 2793,
  'trueDamageDealt': 624,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 7186,
  'magicDamageDealtToChampions': 5902,
  'physicalDamageDealtToChampions': 826,
  'trueDamageDealtToChampions': 458,
  'totalHeal': 3710,
  'totalUnitsHealed': 3,
  'damageSelfMitigated': 2448,
  'damageDealtToObjectives': 404,
  'damageDealtToTurrets': 404,
  'visionScore': 29,
  'timeCCingOthers': 15,
  'totalDamageTaken': 7867,
  'magicalDamageTaken': 4251,
  'physicalDamageTaken': 3510,
  'trueDamag

In [216]:
stats_df = pd.concat([pd.DataFrame(s,index=[0]) for s in stats])
stats_df
# Finally -- able to create a new dataframe for just the stats category which holds a whopping 102 columns of data
# for all 10 participants of the match.

,participantId,win,item0,item1,item2,item3,item4,item5,item6,kills,...,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2
0,1,False,3067,3853,2065,1082,3158,3108,3364,3,...,0,8347,0,0,0,8200,8300,5007,5008,5002
0,2,False,3111,3053,6631,2055,1054,0,3340,1,...,0,8451,180,0,0,8000,8400,5005,5008,5003
0,3,False,1082,6656,3165,3158,1056,4629,3340,5,...,0,8345,3,0,0,8400,8300,5008,5008,5002
0,4,False,6672,3047,2055,3044,1028,0,3364,2,...,0,8135,853,4,0,8000,8100,5005,5008,5003
0,5,False,6691,1036,3158,3042,0,0,3340,1,...,0,8135,1101,5,0,8200,8100,5007,5008,5002
0,6,True,2065,2031,3857,2055,3009,1057,3364,0,...,0,8352,199,1120,31,8400,8300,5008,5008,5002
0,7,True,3158,3748,0,1055,1028,3078,3340,3,...,0,8345,3,0,0,8400,8300,5005,5008,5003
0,8,True,2033,3152,2421,3041,3100,3020,3364,8,...,0,8352,355,1639,1347,8100,8300,5008,5008,5002
0,9,True,1055,3094,1018,3006,3095,6671,3363,9,...,0,8135,2210,5,0,8000,8100,5005,5008,5002
0,10,True,3020,3157,3041,1028,6653,0,3364,5,...,0,8275,19,0,0,8100,8200,5008,5008,5002


In [217]:
pd.set_option('display.max_columns', 105)
# setting the max_columns so we can actually see the data that Riot API holds within this nested nested dict
stats_df

,participantId,win,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,inhibitorKills,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,wardsPlaced,wardsKilled,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,combatPlayerScore,objectivePlayerScore,totalPlayerScore,totalScoreRank,playerScore0,playerScore1,playerScore2,playerScore3,playerScore4,playerScore5,playerScore6,playerScore7,playerScore8,playerScore9,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2
0,1,False,3067,3853,2065,1082,3158,3108,3364,3,3,4,0,1,0,1083,0,0,0,0,0,25604,22187,2793,624,0,7186,5902,826,458,3710,3,2448,404,404,29,15,7867,4251,3510,105,6784,6150,0,0,27,0,0,0,91,11,2,0,17,2,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8214,502,632,0,8226,250,406,0,8210,0,0,0,8236,24,0,0,8345,3,0,0,8347,0,0,0,8200,8300,5007,5008,5002
0,2,False,3111,3053,6631,2055,1054,0,3340,1,5,3,0,1,0,455,0,0,0,0,0,142124,0,104334,37789,0,11335,0,10049,1285,699,1,17354,3367,3367,18,20,20637,8271,11004,1362,8749,8625,1,0,177,8,8,0,278,14,6,0,12,1,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8010,287,0,0,9111,247,80,0,9105,20,20,0,8299,408,0,0,8473,763,0,0,8451,180,0,0,8000,8400,5005,5008,5003
0,3,False,1082,6656,3165,3158,1056,4629,3340,5,4,4,3,2,2,951,1,0,0,0,0,108199,90720,15799,1680,0,16540,15109,1430,0,9028,1,11623,4903,4502,20,41,22738,11855,9928,954,11321,10850,1,0,169,2,0,1,378,14,2,0,9,1,True,False,True,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8437,534,388,0,8446,1679,0,0,8473,775,0,0,8451,188,0,0,8347,0,0,0,8345,3,0,0,8400,8300,5008,5008,5002
0,4,False,6672,3047,2055,3044,1028,0,3364,2,8,4,2,1,1,312,0,0,0,0,0,97464,9035,70884,17544,281,9829,498,7252,2079,8463,1,12134,22639,0,27,6,25857,11021,13965,870,7290,6875,1,0,24,95,72,0,150,11,7,0,5,8,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8010,400,0,0,9111,150,120,0,9105,16,30,0,8014,189,0,0,8143,275,0,0,8135,853,4,0,8000,8100,5005,5008,5003
0,5,False,6691,1036,3158,3042,0,0,3340,1,5,5,0,1,0,471,0,0,0,0,0,81101,19120,60804,1176,418,17014,5292,10545,1176,1643,2,3687,947,947,11,17,12701,5484,5577,1639,7871,7500,0,0,152,0,0,0,269,12,0,0,8,4,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8229,1973,0,0,8226,250,1050,0,8210,1,0,0,8237,895,0,0,8126,1353,0,0,8135,1101,5,0,8200,8100,5007,5008,5002
0,6,True,2065,2031,3857,2055,3009,1057,3364,0,1,16,0,0,0,1357,0,0,0,0,0,21454,8627,6635,6191,0,5849,4610,760,478,2962,1,10114,2228,1238,45,27,13200,5302,6987,909,6571,5490,0,0,27,0,0,0,62,13,6,0,20,1,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8439,106,0,0,8446,831,0,0,8473,600,0,0,8242,24,0,0,8345,3,0,0,8352,199,1120,31,8400,8300,5008,5008,5002
0,7,True,3158,3748,0,1055,1028,3078,3340,3,5,7,2,1,1,291,0,0,0,0,0,104538,829,89840,13868,0,12730,829,8581,3318,5070,1,16244,5988,3371,22,13,21508,7120,12666,1721,9434,8558,2,0,159,8,4,4,191,14,3,0,10,1,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8437,753,560,0,8401,569,0,0,8444,1093,0,0,8451,184,0,0,8304,12,0,0,8345,3,0,0,8400,8300,5005,5008,5003
0,8,True,2033,3152,2421,3041,3100,3020,3364,8,1,9,7,1,1,1363,0,0,0,0,0,138322,119455,17305

# One way of viewing the data

###### manually grabbing the data from the riot api and creating multiple dataframes to hold all of that information
Lots of factors make this inefficient including the amount of ids that this generates for the information that you'd like as well as the rate limiter that api's have to limit the amount of traffic. Especially the riot api because it inhibits personal projects from grabbing large amounts of data.